In [ ]:
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer
import warnings
import math
import matplotlib.pyplot as plt

In [ ]:
#nltk.download("stopwords")


In [ ]:
quora_train = pandas.read_csv("data/train.csv")

In [ ]:
print (type(quora_train))
print(quora_train.head())



In [ ]:
# Editing questions with NLTK package

def stem_stopwords_punctuation_case (phrase):
    """
    Receives a phrase and returns the same phrase stemmed, lowercase phrase without stopwords
    :param package_name: String. A phrase.
    :return: String. Stemmed, lowercase phrase without stopwords
    """

    stemmer = SnowballStemmer("english")
    stopwords = nltk.corpus.stopwords.words('english')
    translator = str.maketrans('', '', string.punctuation)
    
    if type(phrase) is float:
        if math.isnan(phrase):
            return ("")
    
    phrase = phrase.translate(translator) #removing punctuation
    phrase = phrase.lower() #removing word case
    
    stemmed = []
    words = phrase.split(" ")
    for word in words:
        if word not in stopwords:
            stemmed.append(stemmer.stem(word))
    
    stemmed = ' '.join(stemmed)
    return stemmed


stem_stopwords_punctuation_case(
    "What is the step by step guide to invest in share market in india?")



In [ ]:
#quora_train = quora_train.head() #Only for test. Should be removed later

#This is causing warning to appear, should be fixed.
quora_train["question1_edited"] = quora_train["question1"].apply(lambda x: stem_stopwords_punctuation_case(x))

In [ ]:
quora_train["question2_edited"] = quora_train["question2"].apply(lambda x: stem_stopwords_punctuation_case(x))

In [ ]:
print (quora_train.head())

In [ ]:
def calculate_common_percentage(df):
    """
    Receives the initial data frame and adds  the colunms "num_words_common", "num_words_total" and "common_percentage"
    :param package_name: Data frame train.csv from the Kaggle website
    :return: Data frame with added colunms "num_words_common", "num_words_total" and "common_percentage"
    """

    num_words_common = []
    num_words_total = []

    for line in range(0,len(df)):
        count_total = 0
        count_common = 0
        for word in df["question1_edited"][line].split(" "):
            if word in df["question2_edited"][line]:
                count_common = count_common+1
            count_total = count_total+1
        num_words_common.append(count_common) 
        num_words_total.append(count_total)

    num_words_common = pandas.Series(num_words_common)
    num_words_total = pandas.Series(num_words_total)

    df["num_words_common"] = num_words_common.values
    df["num_words_total"] = num_words_total.values
    df["common_percentage"] = df["num_words_common"]/df["num_words_total"]

    return (df)

quora_train = calculate_common_percentage(quora_train)

print (quora_train.head())

In [ ]:
#"num_words_common", "num_words_total" and "common_percentage"

plt.figure()
#plt.boxplot(quora_train["common_percentage"],quora_train["is_duplicate"])


quora_train.boxplot(column='common_percentage', by='is_duplicate')
plt.suptitle('')

axes = plt.gca()
axes.set_ylim([-0.2,1.1])
plt.title("Common word percentage and is_duplicate")

plt.show()

In [ ]:
def make_basic_features(df):
    df["len_q1"] = df.question1.apply(lambda x: len(str(x)))
    df["len_q2"] = df.question2.apply(lambda x: len(str(x)))
    df["diff_len"] = df.len_q1 - df.len_q2
    df["len_char_q1"] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))
    df["len_char_q2"] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))
    df["len_word_q1"] = df.question1.apply(lambda x: len(str(x).split()))
    df["len_word_q2"] = df.question2.apply(lambda x: len(str(x).split()))
    df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split())))), axis=1)